Warp Radolan data to GTIFF 

In [16]:
#Import Modules
import wradlib as wrl
import numpy as np
import warnings
warnings.filterwarnings('ignore')

#Working dir
wdir = "/home/elisabeth/Dokumente/master/radolan_data/raw/"
file = "raa01-sf_10000-1701012350-dwd---bin"
#Read the original data: data = 2d Array(row,col) and metadata (productType, datetime,nodataflag)
#Origin in lower left corner
data_raw, meta  = wrl.io.read_radolan_composite(wdir + file)

#Round data to 2 decimal places to save memory
data_raw_s = np.around(data_raw, decimals = 2, out = None)

# Create Spatail Reference (Radolan projection)
proj_osr = wrl.georef.create_osr("dwd-radolan")

#Load EPSG 3857 
epsg_3857 = wrl.georef.epsg_to_osr(3857)

# Get projected RADOLAN coordinates for corner definition
#origin in lower left corner
xy_raw = wrl.georef.get_radolan_grid(900, 900)

#flip the data and the coords up-down (from lower left to upper left corner)
data, xy = wrl.georef.set_raster_origin(data_raw, xy_raw, 'upper')

#Create raster dataset: Geotiff with radolan projection
ds = wrl.georef.create_raster_dataset(data, xy, projection=proj_osr)

#Reproject dataset to EPSG 3857
ds2 = wrl.georef.reproject_raster_dataset(ds,projection_target=epsg_3857, spacing=1000)
#Bug appears it will fixed in future https://groups.google.com/forum/#!topic/wradlib-users/-dvRhDCjgV0
#This line is needed until fixed
ds2.SetProjection(epsg_3857.ExportToWkt())

#write raster dataset
filename = file[15:25] + ".tif"
wrl.io.write_raster_dataset(wdir + filename, ds2, "GTiff")
#

dest_src available


In [21]:
#Import Modules
import wradlib as wrl
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import os 

#Working dir paths must be clean otherwise you have to make list comprehension that only bin files in files list
outdir ="/media/elisabeth/Seagate 1TB/radolan_downloads/"
wdir = "/home/elisabeth/Dokumente/master/radolan_data/raw/radolan_downloads/"
out = "out"
#Change to Workingdir
os.chdir(outdir)

if not os.path.exists(out):
    os.mkdir(out)

os.chdir(wdir)
paths = os.listdir()

# Create Spatail Reference (Radolan projection)
proj_osr = wrl.georef.create_osr("dwd-radolan")

#Load EPSG 3857 
epsg_3857 = wrl.georef.epsg_to_osr(3857)

# Get projected RADOLAN coordinates for corner definition
#origin in lower left corner
xy_raw = wrl.georef.get_radolan_grid(900, 900)


for path in paths:
    os.chdir(path)
    print("Workingdir: ", os.getcwd())
    files = os.listdir()
    for file in files:
        print("Load: ",file ," data, georeferencing, reprojection and translate to Gtif...")
        data_raw, meta  = wrl.io.read_radolan_composite(os.getcwd() + "/"+ file)
        #flip the data and the coords up-down (from lower left to upper left corner)
        data, xy = wrl.georef.set_raster_origin(data_raw, xy_raw, 'upper')
        #Create raster dataset: Geotiff with radolan projection
        ds = wrl.georef.create_raster_dataset(data, xy, projection=proj_osr)
        #Reproject dataset to EPSG 3857
        ds2 = wrl.georef.reproject_raster_dataset(ds,projection_target=epsg_3857, spacing=1000)
        #Bug appears it will fixed in future https://groups.google.com/forum/#!topic/wradlib-users/-dvRhDCjgV0
        #This line is needed until fixed
        ds2.SetProjection(epsg_3857.ExportToWkt())
        #write raster dataset
        filename = file[15:25] + ".tif"
        wrl.io.write_raster_dataset(outdir + out + "/" + filename, ds2, "GTiff")
        print("Save", filename, "to", os.getcwd())
    os.chdir("../")

        

Workingdir:  /home/elisabeth/Dokumente/master/radolan_data/raw/radolan_downloads/2019
Load:  raa01-sf_10000-1906282350-dwd---bin  data, georeferencing, reprojection and translate to Gtif...
dest_src available
Save 1906282350.tif to /home/elisabeth/Dokumente/master/radolan_data/raw/radolan_downloads/2019
Load:  raa01-sf_10000-1905042350-dwd---bin  data, georeferencing, reprojection and translate to Gtif...
dest_src available
Save 1905042350.tif to /home/elisabeth/Dokumente/master/radolan_data/raw/radolan_downloads/2019
Load:  raa01-sf_10000-1907292350-dwd---bin  data, georeferencing, reprojection and translate to Gtif...
dest_src available
Save 1907292350.tif to /home/elisabeth/Dokumente/master/radolan_data/raw/radolan_downloads/2019
Load:  raa01-sf_10000-1905302350-dwd---bin  data, georeferencing, reprojection and translate to Gtif...
dest_src available
Save 1905302350.tif to /home/elisabeth/Dokumente/master/radolan_data/raw/radolan_downloads/2019
Load:  raa01-sf_10000-1907082350-dwd--

Save 1903132350.tif to /home/elisabeth/Dokumente/master/radolan_data/raw/radolan_downloads/2019
Load:  raa01-sf_10000-1901052350-dwd---bin  data, georeferencing, reprojection and translate to Gtif...
dest_src available
Save 1901052350.tif to /home/elisabeth/Dokumente/master/radolan_data/raw/radolan_downloads/2019
Load:  raa01-sf_10000-1904182350-dwd---bin  data, georeferencing, reprojection and translate to Gtif...
dest_src available
Save 1904182350.tif to /home/elisabeth/Dokumente/master/radolan_data/raw/radolan_downloads/2019
Load:  raa01-sf_10000-1902162350-dwd---bin  data, georeferencing, reprojection and translate to Gtif...
dest_src available
Save 1902162350.tif to /home/elisabeth/Dokumente/master/radolan_data/raw/radolan_downloads/2019
Load:  raa01-sf_10000-1903212350-dwd---bin  data, georeferencing, reprojection and translate to Gtif...
dest_src available
Save 1903212350.tif to /home/elisabeth/Dokumente/master/radolan_data/raw/radolan_downloads/2019
Load:  raa01-sf_10000-190521

Save 1906242350.tif to /home/elisabeth/Dokumente/master/radolan_data/raw/radolan_downloads/2019
Load:  raa01-sf_10000-1904282350-dwd---bin  data, georeferencing, reprojection and translate to Gtif...
dest_src available
Save 1904282350.tif to /home/elisabeth/Dokumente/master/radolan_data/raw/radolan_downloads/2019
Load:  raa01-sf_10000-1903042350-dwd---bin  data, georeferencing, reprojection and translate to Gtif...
dest_src available
Save 1903042350.tif to /home/elisabeth/Dokumente/master/radolan_data/raw/radolan_downloads/2019
Load:  raa01-sf_10000-1901092350-dwd---bin  data, georeferencing, reprojection and translate to Gtif...
dest_src available
Save 1901092350.tif to /home/elisabeth/Dokumente/master/radolan_data/raw/radolan_downloads/2019
Load:  raa01-sf_10000-1904232350-dwd---bin  data, georeferencing, reprojection and translate to Gtif...
dest_src available
Save 1904232350.tif to /home/elisabeth/Dokumente/master/radolan_data/raw/radolan_downloads/2019
Load:  raa01-sf_10000-190621

Save 1901202350.tif to /home/elisabeth/Dokumente/master/radolan_data/raw/radolan_downloads/2019
Load:  raa01-sf_10000-1906052350-dwd---bin  data, georeferencing, reprojection and translate to Gtif...
dest_src available
Save 1906052350.tif to /home/elisabeth/Dokumente/master/radolan_data/raw/radolan_downloads/2019
Load:  raa01-sf_10000-1907182350-dwd---bin  data, georeferencing, reprojection and translate to Gtif...
dest_src available
Save 1907182350.tif to /home/elisabeth/Dokumente/master/radolan_data/raw/radolan_downloads/2019
Load:  raa01-sf_10000-1904192350-dwd---bin  data, georeferencing, reprojection and translate to Gtif...
dest_src available
Save 1904192350.tif to /home/elisabeth/Dokumente/master/radolan_data/raw/radolan_downloads/2019
Load:  raa01-sf_10000-1907232350-dwd---bin  data, georeferencing, reprojection and translate to Gtif...
dest_src available
Save 1907232350.tif to /home/elisabeth/Dokumente/master/radolan_data/raw/radolan_downloads/2019
Load:  raa01-sf_10000-190312

Save 1903192350.tif to /home/elisabeth/Dokumente/master/radolan_data/raw/radolan_downloads/2019
Load:  raa01-sf_10000-1902252350-dwd---bin  data, georeferencing, reprojection and translate to Gtif...
dest_src available
Save 1902252350.tif to /home/elisabeth/Dokumente/master/radolan_data/raw/radolan_downloads/2019
Load:  raa01-sf_10000-1907282350-dwd---bin  data, georeferencing, reprojection and translate to Gtif...
dest_src available
Save 1907282350.tif to /home/elisabeth/Dokumente/master/radolan_data/raw/radolan_downloads/2019
Load:  raa01-sf_10000-1907202350-dwd---bin  data, georeferencing, reprojection and translate to Gtif...
dest_src available
Save 1907202350.tif to /home/elisabeth/Dokumente/master/radolan_data/raw/radolan_downloads/2019
Load:  raa01-sf_10000-1902112350-dwd---bin  data, georeferencing, reprojection and translate to Gtif...
dest_src available
Save 1902112350.tif to /home/elisabeth/Dokumente/master/radolan_data/raw/radolan_downloads/2019
Load:  raa01-sf_10000-190706

Save 1902082350.tif to /home/elisabeth/Dokumente/master/radolan_data/raw/radolan_downloads/2019
Load:  raa01-sf_10000-1907112350-dwd---bin  data, georeferencing, reprojection and translate to Gtif...
dest_src available
Save 1907112350.tif to /home/elisabeth/Dokumente/master/radolan_data/raw/radolan_downloads/2019
Load:  raa01-sf_10000-1902072350-dwd---bin  data, georeferencing, reprojection and translate to Gtif...
dest_src available
Save 1902072350.tif to /home/elisabeth/Dokumente/master/radolan_data/raw/radolan_downloads/2019
Load:  raa01-sf_10000-1902092350-dwd---bin  data, georeferencing, reprojection and translate to Gtif...
dest_src available
Save 1902092350.tif to /home/elisabeth/Dokumente/master/radolan_data/raw/radolan_downloads/2019
Load:  raa01-sf_10000-1902272350-dwd---bin  data, georeferencing, reprojection and translate to Gtif...
dest_src available
Save 1902272350.tif to /home/elisabeth/Dokumente/master/radolan_data/raw/radolan_downloads/2019
Load:  raa01-sf_10000-190401

ValueError: time data 'a01-sf0-1200' does not match format '%d%H%M%m%y%S'